# Instagram Scraper usando pygram

In [195]:
# Se importan las librerias
from pygram import PyGram
import pandas as pd
from datetime import datetime
import time
from pathlib import Path
import numpy as np
import random

pygram = PyGram()

Se seleccionan los usuarios a los cuales se les van a buscar los posts y se establece la ruta de la carpeta de trabajo

In [196]:
# usernames
usernames = ['offcorss', 'epk', 'babyfreshoficial', 'politokids']
# folder path
folder_path = r'C:\Users\DavidCordoba\Google Drive\Estudio\Cursos\2020. DS4A\Project\Instagram_Scraper\proyect_DS4A-master'

# Publicaciones de usuarios

## Función para extraer las publicaciones del usuario seleccionados

Se crea una función para extraer la información de un solo usuario.

In [197]:
def get_posts_df(username, limit = 100):
    """ Extrae los posts del usuario y los convierta en un pandas dataframe.
    RETORNA:
    pandas.DataFrame: pandas.DataFrame con la información de los posts solicitados del usuario seleccionado.
    """
    
    # Devuelve una lista de diccionarios
    posts = list(pygram.get_posts(username, limit = limit))
    print(len(posts), ' posts collected from ', username)

    # Convierte la lista de diccionarios en pd dataframe
    posts_df = pd.DataFrame(posts)
    # Adiciona en columna date
    posts_df['date'] = posts_df.timestamp.apply(lambda x: datetime.fromtimestamp(x))

    return posts_df

## Función para extraer las publicaciones todos los usuarios

Se crea otra función para extraer la información de múltiples usuarios.

In [198]:
def get_posts_from_usernames(usernames, limit = 100):
    """ Crea una dataframe con un número especifico de posts de todos los usernames ingresados.
    RETORNA:
    pandas.DataFrame: pandas.DataFrame con la información de los posts solicitados de los usuarios seleccionados.
    """
    
    # Se crea una lista de dataframes con los dataframes de cada user
    posts_df = []
    for user in usernames:
        posts_df.append(get_posts_df(user, limit = limit))
        
    # Se concatenan los dataframes en uno solo
    posts_df = pd.concat(posts_df)
    
    return posts_df

## Actualización o creación de base de datos de publicaciones

Se busca (en caso de que exista), una base de datos con información de publiaciones anteriores y se actualiza con la nueva información encontrada (si la publicación ya estaba en la base de datos, se actualizan sus parámetros tales cómo comentarios, likes, vistas, etc y si la publicación no estaba en la base de datos entonces se añade).

In [199]:
# Se llaman las funciones para recolectar la última información de posts de los usuarios
posts_df = get_posts_from_usernames(usernames, limit = 10)

try:
    # Se lee la base de datos
    posts_database = pd.read_csv(Path(folder_path)/'posts_database.csv')
    # Se actualiza la base de datos
    posts_database = pd.concat([posts_database, posts_df])
    posts_database['id'] = posts_database['id'].astype('int64')
    posts_database.drop_duplicates(subset = 'id', keep = 'last', inplace = True)
    # Se guarda la base de datos
    posts_database.to_csv(Path(folder_path)/'posts_database.csv', index = False)
    
except:
    # Se crea y guarda la base de datos
    posts_df.to_csv(Path(folder_path)/'posts_database.csv', index = False)

10  posts collected from  offcorss
10  posts collected from  epk
10  posts collected from  babyfreshoficial
10  posts collected from  politokids


In [200]:
# Esto solo nos sirve para dar una idea de la data, no realiza ninguna alteración a la data
posts_database = pd.read_csv(Path(folder_path)/'posts_database.csv')

print(posts_database.shape)
print(len(posts_database.reset_index()['id'].unique()))
posts_database.describe()

(20060, 12)
20060


,id,comments_count,timestamp,likes_count,author,video_views_count
count,2.006000e+04,20060.000000,2.006000e+04,20060.000000,2.006000e+04,2275.000000
mean,1.521979e+18,7.972084,1.495654e+09,299.516052,2.287647e+08,3576.322637
std,5.342097e+17,28.146271,6.368277e+07,405.083033,2.025129e+07,6839.484191
min,2.578409e+17,0.000000,1.344957e+09,0.000000,2.080347e+08,0.000000
25%,1.103811e+18,0.000000,1.445805e+09,85.000000,2.142773e+08,0.000000
50%,1.564603e+18,3.000000,1.500735e+09,182.000000,2.221308e+08,1262.000000
75%,1.946751e+18,8.000000,1.546291e+09,360.000000,2.605503e+08,5077.000000
max,2.416338e+18,1578.000000,1.602270e+09,11051.000000,2.605503e+08,117171.000000


# Comentarios de publicaciones

## Función para extraer los comentarios de una publicación y actualizar la base de datos

Esta función tiene cómo objetivo extraer los comentarios de una sola publicación y con esta información actualizar la base de datos en csv, esto debido a que la forma de extraer los comentarios tiende a fallar con facilidad entonces para no realizar solicitudes erroneas y evitar la perdida de la información. La función busca en la base de datos de comentarios que publicación todavia no se ha scrapeado nunca y trata de realizar el proceso.

In [201]:
def get_comments_df(posts_database, comments_database, limit = 10000):
    """ Actualiza el dataframe de comentarios con los comentarios de una de las publicaciónes que no se haya scrapeado.
    RETORNA:
    pandas.DataFrame: pandas.DataFrame de los comentarios actualizado y el estado de la transacción (Exitosa o fallida).
    """
    
    # Se busca un id que no tenga comentarios
    post_id = random.choice(list(comments_database[pd.isna(comments_database['id'])]['post_id']))
    # Se encuentra el post de ese id en df
    post_df = posts_database[posts_database['id'] == post_id]
    # Se pasa el post df a una estructura de diccionario
    post_dict = post_df.to_dict(orient='index')
    post_dict = list(post_dict.values())[0]
    
    try:
          
        # Se buscan los comentarios de ese post y devuelve una lista de diccionarios
        comments = list(pygram.get_comments(post_dict, limit = limit))
        print(len(comments), ' comments collected from the post with ID: ', post_id)
    
        # Convierte la lista de diccionarios en pd dataframe
        comments_df = pd.DataFrame(comments)
        # Adiciona en columna date
        comments_df['date'] = comments_df.timestamp.apply(lambda x: datetime.fromtimestamp(x))
        # Impongo esquema
        comments_dtypes = {'id': 'object',
                           'text': 'object',
                           'timestamp': 'float64',
                           'author': 'object',
                           'post_id': 'object',
                           'date': 'datetime64[ns]'}
        for key, value in comments_dtypes.items():
            comments_df[key] = comments_df[key].astype(value)
            
        # Elimino la fila que tiene el post_id scrapeado
        comments_database.drop(comments_database.index[comments_database['post_id'] == post_id].tolist(), inplace = True)
        # Agrego las filas con los comentarios scrapeados y Concateno ambos dataframes
        comments_database = pd.concat([comments_database, comments_df])
        # Se guarda la base de datos
        comments_database.to_csv(Path(folder_path)/'comments_database.csv', index = False)  
        success = True
        
    except:
        
        print('There was an error when running the pygram library, the limit was probably exceeded')
        success = False
        
    return comments_database, success

## Función para actualizar la base de datos de los comentarios con los post ids nuevos

En esta función se utiliza la base de datos de publicaciones para mirar que nuevas publicaciones hay que la base de datos de comentarios no esté teniendo en cuenta y actualiza el dataframe de comentarios únicamente con el post id de las publicaciones que deben ser posteriormente scrapeadas.

In [202]:
def update_post_ids_in_comments_database():
    """ Actualiza el dataframe de comentarios los post ids nuevos que tenga la base de datos de publicaciones.
    RETORNA:
    pandas.DataFrame: pandas.DataFrame de los comentarios actualizado con los nuevos post ids.
    """

    # Se lee la base de datos de los posts y se impone el esquema
    posts_database = pd.read_csv(Path(folder_path)/'posts_database.csv')
    posts_dtypes = {'id': 'object',
                    'shortcode': 'object',
                    'comments_disabled': 'bool',
                    'comments_count': 'int64',
                    'timestamp': 'float64',
                    'display_url': 'object',
                    'likes_count': 'int64',
                    'author': 'object',
                    'is_video': 'bool',
                    'caption': 'object',
                    'video_views_count': 'float64',
                    'date': 'datetime64[ns]'}
    for key, value in posts_dtypes.items():
        posts_database[key] = posts_database[key].astype(value)

    # Se trata de leer la base de datos de los comentarios y sino se buscan los comentarios de una publicacion cualquiera para crear la estructura
    try:

        # Se lee la base de datos de los comentarios y se impone el formato
        comments_database = pd.read_csv(Path(folder_path)/'comments_database.csv')
        comments_dtypes = {'id': 'object',
                           'text': 'object',
                           'timestamp': 'float64',
                           'author': 'object',
                           'post_id': 'object',
                           'date': 'datetime64[ns]'}
        for key, value in comments_dtypes.items():
            comments_database[key] = comments_database[key].astype(value)

        # Adiciono todos los id de posts que no esten en el comments_df
        # Creo un df con todos los posts id que no estan en comments_database
        comments_df = pd.DataFrame(columns = ['id', 'text', 'timestamp', 'author', 'post_id', 'date'])
        comments_df['post_id'] = [post_id for post_id in list(posts_database[posts_database['comments_count'] > 0]['id']) if post_id not in list(comments_database['post_id'])]
        # Se impone el formato
        for key, value in comments_dtypes.items():
            comments_df[key] = comments_df[key].astype(value)
        # Concateno ambos dataframes
        comments_database = pd.concat([comments_database, comments_df])
        # Se guarda la base de datos
        comments_database.to_csv(Path(folder_path)/'comments_database.csv', index = False)  

    except:

        # Creo un df vacio con las columnas respectivas y le impongo el esquema
        comments_database = pd.DataFrame(columns = ['id', 'text', 'timestamp', 'author', 'post_id', 'date'])
        # Adiciono todos los id de posts que no esten en el comments_df
        comments_database['post_id'] = list(posts_database[posts_database['comments_count'] > 0]['id'])
        # Impongo el formato del esquema
        comments_dtypes = {'id': 'object',
                           'text': 'object',
                           'timestamp': 'float64',
                           'author': 'object',
                           'post_id': 'object',
                           'date': 'datetime64[ns]'}
        for key, value in comments_dtypes.items():
            comments_database[key] = comments_database[key].astype(value)
        # Se crea y guarda la base de datos
        comments_database.to_csv(Path(folder_path)/'comments_database.csv', index = False)

#     print(comments_database.info(), '\n')
#     try:
#         print(comments_df.info(), '\n')
#     except:
#         pass
    
    return posts_database, comments_database

## Función para completar la base de datos de comentarios con todos los comentarios faltantes

Esta función explora la base de datos de comentarios y busca todos los post ids que no tengan comentarios y recorre todos estos buscando sus respectivos comentarios en Instragram hasta que la libreria de pygram falla y entonces se detiene.

In [203]:
def fill_comments_database(limit_posts = -1, limit_comments = 10000, wait = 5):
    """ Trata de completar todos los comentarios faltantes de la base de datos de comentarios haciendo uso de los post ids.
    RETORNA:
    None.
    """
    
    # Actualizo las bases de datos con los post ids
    posts_database, comments_database = update_post_ids_in_comments_database()
    # Calculo el máximo e posts a iterar, si es -1 entonces todos los faltantes y si es diferente a -1 entonces ese número.
    if limit_posts == -1:
        limit_posts = len(comments_database[pd.isna(comments_database['id'])]['post_id'])
    
    # Voy a través de todos los post id que no tienen comentarios y trato de completarlos
    for i in range(limit_posts):
        
        time.sleep(wait)
        # Actualizo un comentario de la base de datos
        comments_database, success = get_comments_df(posts_database, comments_database, limit = limit_comments)
        if not success:
            break
    
    return None

## Función para actualizar los comentarios de las publicaciones más recientes

Esta función sirve para buscar las publicaciones más recientes, hacer scraping de sus comentarios y actualizar los nuevos comentarios en la base de datos de comentarios.

In [204]:
def update_comments_database(limit_posts = 100, limit_comments = 10000, wait = 5):
    """ Trata de actualizar los comentarios de las publicaciones más recientes.
    RETORNA:
    None.
    """
    
    posts_database, comments_database = update_post_ids_in_comments_database()

    # Se buscan los 100 posts más actuales
    post_ids = list(posts_database.sort_values(by = ['timestamp'], ascending = False)['id'])[0:limit_posts] #revisar codigo
    # Se encuentran los posts de esos ids
    posts_df = posts_database[posts_database['id'].isin(post_ids)]
    # Se pasa el post df a una estructura de diccionario
    posts_dict = posts_df.to_dict(orient = 'index')
    posts_dict = list(posts_dict.values())

    for post_dict in posts_dict:

        time.sleep(wait)

        try:

            # Se buscan los comentarios de ese post y devuelve una lista de diccionarios
            comments = list(pygram.get_comments(post_dict, limit = limit_comments))
            print(len(comments), ' comments collected from the post with ID: ', post_dict['id'])

            # Convierte la lista de diccionarios en pd dataframe
            comments_df = pd.DataFrame(comments)
            # Adiciona en columna date
            comments_df['date'] = comments_df.timestamp.apply(lambda x: datetime.fromtimestamp(x))
            # Impongo esquema
            comments_dtypes = {'id': 'object',
                               'text': 'object',
                               'timestamp': 'float64',
                               'author': 'object',
                               'post_id': 'object',
                               'date': 'datetime64[ns]'}
            for key, value in comments_dtypes.items():
                comments_df[key] = comments_df[key].astype(value)

            # Agrego las filas con los comentarios scrapeados y Concateno ambos dataframes y elimino los comentarios repetidos
            comments_database = pd.concat([comments_database, comments_df])
            comments_database['id'] = comments_database['id'].astype('float64')
            comments_database.drop_duplicates(subset = 'id', keep = 'last', inplace = True)
            comments_database['id'] = comments_database['id'].astype('object')
            # Se guarda la base de datos
            comments_database.to_csv(Path(folder_path)/'comments_database.csv', index = False)  
            success = True

        except:

            print('There was an error when running the pygram library, the limit was probably exceeded')
            success = False
            break
            
    return None

## Completado de los comentarios de las publicaciones

Se utiliza la función de completado de comentarios para llenar la base de datos de comentarios. Si se desea intentar completar toda la base de datos utilziar el parámetro limit_posts = -1.

In [205]:
fill_comments_database(limit_posts = -1, limit_comments = 10000, wait = 5)

8  comments collected from the post with ID:  1659836777407961135
1  comments collected from the post with ID:  386855397782788346
7  comments collected from the post with ID:  1827005512294654315
4  comments collected from the post with ID:  1293106371088184997
4  comments collected from the post with ID:  2022664947184893292
6  comments collected from the post with ID:  2386742890319051782
7  comments collected from the post with ID:  2078004015858105722
3  comments collected from the post with ID:  2127245003429043782
1  comments collected from the post with ID:  1883552776207331272
3  comments collected from the post with ID:  2022685204482523337
1  comments collected from the post with ID:  1016142495494705953
11  comments collected from the post with ID:  1718474990297775788
0  comments collected from the post with ID:  1303718315532886380
There was an error when running the pygram library, the limit was probably exceeded


## Actualización de base de datos de comentarios

Se utiliza la función de actualización de comentarios para buscar las publicaciones más recientes y actualizar sus comentarios.

In [206]:
update_comments_database(limit_posts = 2, limit_comments = 10000, wait = 5)

20  comments collected from the post with ID:  2416323704633079000
7  comments collected from the post with ID:  2416337884922791971
